In [2]:
import pandas as pd
import _job_count_calculation

call_df = _job_count_calculation.make_job_count_df(path="../data/run_results.csv",
                      params_path="../data/run_params_used.csv")

call_df['timestamp_dt'] = pd.to_datetime(call_df['timestamp_dt'])
call_df['month_start'] = call_df['timestamp_dt'].dt.to_period('M').dt.to_timestamp()

jobs_counts_by_callsign_monthly_sim = (
    call_df
    .groupby(['run_number', 'month_start', 'callsign', 'callsign_group', 'vehicle_type'])['P_ID']
    .count().reset_index().rename(columns={'P_ID': 'jobs'})
    )

jobs_counts_by_callsign_monthly_sim = jobs_counts_by_callsign_monthly_sim[~jobs_counts_by_callsign_monthly_sim['callsign'].isna()]

all_combinations = pd.MultiIndex.from_product([
    jobs_counts_by_callsign_monthly_sim["month_start"].unique(),
    jobs_counts_by_callsign_monthly_sim["run_number"].unique(),
    jobs_counts_by_callsign_monthly_sim["callsign"].unique()
], names=["month_start", "run_number", "callsign"])

# Reindex the dataframe to include missing callsigns
jobs_counts_by_callsign_monthly_sim = jobs_counts_by_callsign_monthly_sim.set_index(
    ["month_start", "run_number", "callsign"]
).reindex(all_combinations, fill_value=0).reset_index()

In [3]:
jobs_counts_by_callsign_monthly_sim

,month_start,run_number,callsign,callsign_group,vehicle_type,jobs
0,2025-02-01,1,C70,70.0,car,20
1,2025-02-01,1,C71,71.0,car,8
2,2025-02-01,1,C72,72.0,car,10
3,2025-02-01,1,H70,70.0,helicopter,9
4,2025-02-01,1,H71,0.0,0,0
...,...,...,...,...,...,...
1295,2026-02-01,20,C70,70.0,car,43
1296,2026-02-01,20,C71,71.0,car,10
1297,2026-02-01,20,C72,72.0,car,14
1298,2026-02-01,20,H70,70.0,helicopter,13
